In [22]:
import numpy as np
import numpy.matlib

In [135]:
import math

In [254]:
def fuck_bsas(X, theta, q, order):
    [l, N] = X.shape
    n_clust = 1

    if (len(order) == 0):
        order = np.matrixlib.mat(range(N))
    bel = np.matlib.zeros((1, N), dtype=np.int)
    
    bel[0, order[0,0]] = n_clust
    repre = X[:, order[0,0]]
    
    for i in range(1, N):
        [m1, m2] = repre.shape
        
        tmp = repre - X[:, order[0,i]] * np.matlib.ones((1, m2), dtype=np.int)
        tmp = np.sqrt(np.sum(np.square(tmp), axis=0))
        print(tmp)
        s2 = np.unravel_index(tmp.argmin(), tmp.shape)
        print(s2)
        s1 = tmp[s2]        
        
        if ((s1 > theta) and (n_clust < q)):
            n_clust+=1
            bel[0, order[0,i]] = n_clust
            repre = np.concatenate((repre, X[:, order[0,i]]), axis=1)
        else:
            bel[0, order[0,i]] = s2[1]+1
            tmp = np.sum(np.equal(bel, s2[1]+1))
            repre[:, s2[1]] = ( (tmp - 1)*repre[:, s2[1]] + X[:, order[0,i]] ) / tmp
    return bel, repre

In [246]:
X = np.matrix('2 5; 6 4; 5 3; 2 2; 1 4; 5 4; 3 3; 2 3; 2 4; 8 2; 9 2; 10 2; 11 2; 10 3; 9 1')
X = X.T
q = 15
theta = 2.5
order = np.matrix('8 6 11 1 5 2 3 4 7 10 9 12 13 14 15') - np.matlib.ones_like(order, dtype=np.int)

In [255]:
[bel_2, repre_2] = fuck_bsas(X, theta, q, order)

[[ 3.16227766]]
(0, 0)
[[ 7.07106781  4.47213595]]
(0, 1)
[[ 2.          3.16227766  7.61577311]]
(0, 0)
[[ 1.          4.          8.24621125]]
(0, 0)
[[ 5.          1.          3.60555128]]
(0, 1)
[[ 4.12310563  1.          4.12310563]]
(0, 1)
[[ 2.23606798  3.16227766  7.        ]]
(0, 0)
[[ 2.          2.          6.08276253]]
(0, 0)
[[ 7.07106781  3.16227766  1.        ]]
(0, 2)
[[ 1.41421356  3.16227766  6.32455532]]
(0, 0)
[[ 9.05538514  5.09901951  2.        ]]
(0, 2)
[[ 10.04987562   6.08276253   3.        ]]
(0, 2)
[[ 9.          5.          2.23606798  1.41421356]]
(0, 3)
[[ 8.24621125  4.47213595  1.41421356  1.41421356]]
(0, 2)


In [257]:
bel_2

matrix([[1, 2, 2, 1, 1, 2, 1, 1, 1, 3, 3, 3, 4, 4, 3]])

In [258]:
repre_2

matrix([[ 1,  5,  8, 10],
        [ 3,  3,  1,  2]])

In [113]:
np.concatenate((X[:, 8], np.matlib.ones((1, 2), dtype=np.int).T), axis=1)

matrix([[2, 1],
        [4, 1]])

In [194]:
y = np.matrix([-1, 7.07106781, 4.47213595, 2])
print (y)
miny = np.min(y[0,:])
miny

[[-1.          7.07106781  4.47213595  2.        ]]


-1.0

In [193]:
np.matrix(np.where(y == y.min())).flatten()

matrix([[0, 0]], dtype=int64)

In [165]:
y

matrix([[ 3.16227766,  2.        ,  4.        ]])